**Initial Installation and Set-up**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
! wget https://repo.anaconda.com/miniconda/Miniconda3-py37_4.8.2-Linux-x86_64.sh

In [ ]:
! chmod +x Miniconda3-py37_4.8.2-Linux-x86_64.sh

In [ ]:
! bash ./Miniconda3-py37_4.8.2-Linux-x86_64.sh -b -f -p /usr/local

In [ ]:
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages/')

In [ ]:
!pip install pip 2to3==1.0

In [ ]:
!sudo apt-get install openslide-tools

In [ ]:
!conda env create -n clam -f "/content/gdrive/MyDrive/CLAM-master/CLAM-master/docs/clam.yaml"

In [ ]:
!source activate clam

In [ ]:
!cd /usr/local/envs/clam

In [ ]:
!git clone https://github.com/oval-group/smooth-topk.git

In [ ]:
print(a)

1. After this step go on terminal and type cd smooth-topk
2. python setup.py install

In [ ]:
!pip install opencv-python==4.1.1.26

In [ ]:
!conda install matplotlib=3.1.1=py37h5429711_0

In [ ]:
!conda install ipykernel=5.1.4=py37h39e3cac_0

In [ ]:
!conda install ipython_genutils=0.2.0=py37_0

In [ ]:
!pip install openslide-python==1.1.1

In [ ]:
!conda install h5py=2.10.0

In [ ]:
!pip install pandas

In [ ]:
!pip install torchvision==0.13.0

In [ ]:
!pip install scipy==1.4.1

In [ ]:
!pip install sklearn==0.0

In [ ]:
!pip install tensorboard

In [ ]:
!pip install tensorboardx

In [ ]:
!pip install protobuf==3.10.0

In [ ]:
!pip install future==0.18.2

In [ ]:
!pip install torchmetrics

In [ ]:
print(b)

**Image Segmentation & Patch Creation**

In [ ]:
!python "/content/gdrive/MyDrive/CLAM-master/CLAM-master/create_patches_fp.py" --source "/content/gdrive/MyDrive/DATA_DIRECTORY" --save_dir "/content/gdrive/MyDrive/RESULTS_DIRECTORY" --patch_size 256 --seg --patch --stitch 

**Feature Extraction**

In [ ]:
!CUDA_VISIBLE_DEVICES=0,1 python "/content/gdrive/MyDrive/CLAM-master/CLAM-master/extract_features_fp.py" --data_h5_dir "/content/gdrive/MyDrive/RESULTS_DIRECTORY" --data_slide_dir "/content/gdrive/MyDrive/DATA_DIRECTORY" --csv_path "/content/gdrive/MyDrive/CLAM-master/CLAM-master/process_list_autogen.csv" --feat_dir "/content/gdrive/MyDrive/FEATURES_DIRECTORY" --batch_size 16 --slide_ext .tif

**Train, Validation, Test Split**

In [ ]:
!python /content/gdrive/MyDrive/CLAM-master/CLAM-master/create_splits_seq.py --task task_2_tumor_subtyping --seed 1 --label_frac 0.60 --val_frac 0.20 --test_frac 0.20 --k 5

**Training**

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python "/content/gdrive/MyDrive/CLAM-master/CLAM-master/main.py" --drop_out --early_stopping --lr 2e-4 --k 5 --split_dir "/content/splits/task_2_tumor_subtyping_60" --exp_code task_2_tumor_subtyping_CLAM_50 --weighted_sample --bag_loss ce --inst_loss ce --task task_2_tumor_subtyping --model_type clam_sb --log_data --subtyping --data_root_dir "./gdrive/MyDrive/DATA_ROOT_DIR"

**Evaluation**

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python "/content/gdrive/MyDrive/CLAM-master/CLAM-master/eval.py" --drop_out --k 5 --models_exp_code task_2_tumor_subtyping_CLAM_50_s1 --save_exp_code task_2_tumor_subtyping_CLAM_50_s1_cv --task task_2_tumor_subtyping --model_type clam_sb --results_dir "/content/results" --data_root_dir "/content/gdrive/MyDrive/DATA_ROOT_DIR"

**Plotting Loss Curve**

In [ ]:
import pandas as pd
Losses=pd.read_csv('/content/gdrive/MyDrive/Losses_data.csv')

In [ ]:
Fold_0_Losses= Losses[:52]
Fold_0_Losses.columns = ['Index', 'Epoch', 'training_loss_Fold_0', 'validation_loss_Fold_0']
Fold_0_Losses= Fold_0_Losses.drop(['Index'], axis=1)
Fold_1_Losses= Losses[52:104]
Fold_1_Losses.columns = ['Index', 'Epoch', 'training_loss_Fold_1', 'validation_loss_Fold_1']
Fold_1_Losses= Fold_1_Losses.drop(['Index'], axis=1)
Fold_2_Losses= Losses[104:156]
Fold_2_Losses.columns = ['Index', 'Epoch', 'training_loss_Fold_2', 'validation_loss_Fold_2']
Fold_2_Losses= Fold_2_Losses.drop(['Index'], axis=1)
Fold_3_Losses= Losses[156:209]
Fold_3_Losses.columns = ['Index', 'Epoch', 'training_loss_Fold_3', 'validation_loss_Fold_3']
Fold_3_Losses= Fold_3_Losses.drop(['Index'], axis=1)
Fold_4_Losses= Losses[209:]
Fold_4_Losses.columns = ['Index', 'Epoch', 'training_loss_Fold_4', 'validation_loss_Fold_4']
Fold_4_Losses= Fold_4_Losses.drop(['Index'], axis=1)
Fold_Losses= Fold_0_Losses.merge(Fold_1_Losses,on='Epoch').merge(Fold_2_Losses,on='Epoch').merge(Fold_3_Losses,on='Epoch').merge(Fold_4_Losses,on='Epoch')
Fold_Losses['Train_Loss']= (Fold_Losses['training_loss_Fold_0']+ Fold_Losses['training_loss_Fold_1']+ Fold_Losses['training_loss_Fold_2']+ Fold_Losses['training_loss_Fold_3']+ Fold_Losses['training_loss_Fold_4'])/5
Fold_Losses['Validation_Loss']= (Fold_Losses['validation_loss_Fold_0']+ Fold_Losses['validation_loss_Fold_1']+ Fold_Losses['validation_loss_Fold_2']+ Fold_Losses['validation_loss_Fold_3']+ Fold_Losses['validation_loss_Fold_4'])/5
Fold_Losses

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.title("Training and Validation Loss")
plt.plot(Fold_Losses.Epoch, Fold_Losses.Validation_Loss, label="val")
plt.plot(Fold_Losses.Epoch, Fold_Losses.Train_Loss, label="train")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

**Plotting Errors Curve**

In [ ]:
Errors=pd.read_csv('/content/gdrive/MyDrive/Errors_data.csv')
Errors

In [ ]:
Fold_0_Errors= Errors[:52]
Fold_0_Errors.columns = ['Index', 'Epoch', 'training_error_Fold_0', 'validation_error_Fold_0']
Fold_0_Errors= Fold_0_Errors.drop(['Index'], axis=1)
Fold_1_Errors= Errors[52:104]
Fold_1_Errors.columns = ['Index', 'Epoch', 'training_error_Fold_1', 'validation_error_Fold_1']
Fold_1_Errors= Fold_1_Errors.drop(['Index'], axis=1)
Fold_2_Errors= Errors[104:156]
Fold_2_Errors.columns = ['Index', 'Epoch', 'training_error_Fold_2', 'validation_error_Fold_2']
Fold_2_Errors= Fold_2_Errors.drop(['Index'], axis=1)
Fold_3_Errors= Errors[156:209]
Fold_3_Errors.columns = ['Index', 'Epoch', 'training_error_Fold_3', 'validation_error_Fold_3']
Fold_3_Errors= Fold_3_Errors.drop(['Index'], axis=1)
Fold_4_Errors= Errors[209:]
Fold_4_Errors.columns = ['Index', 'Epoch', 'training_error_Fold_4', 'validation_error_Fold_4']
Fold_4_Errors= Fold_4_Errors.drop(['Index'], axis=1)
Fold_Errors= Fold_0_Errors.merge(Fold_1_Errors,on='Epoch').merge(Fold_2_Errors,on='Epoch').merge(Fold_3_Errors,on='Epoch').merge(Fold_4_Errors,on='Epoch')
Fold_Errors['Train_Error']= (Fold_Errors['training_error_Fold_0']+ Fold_Errors['training_error_Fold_1']+ Fold_Errors['training_error_Fold_2']+ Fold_Errors['training_error_Fold_3']+ Fold_Errors['training_error_Fold_4'])/5
Fold_Errors['Validation_Error']= (Fold_Errors['validation_error_Fold_0']+ Fold_Errors['validation_error_Fold_1']+ Fold_Errors['validation_error_Fold_2']+ Fold_Errors['validation_error_Fold_3']+ Fold_Errors['validation_error_Fold_4'])/5
Fold_Errors

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,5))
plt.title("Training and Validation Errors")
plt.plot(Fold_Errors.Epoch, Fold_Errors.Validation_Error, label="val")
plt.plot(Fold_Errors.Epoch, Fold_Errors.Train_Error, label="train")
plt.xlabel("Epochs")
plt.ylabel("Error")
plt.legend()
plt.show()